# Loading COVID-19 Data to Google Sheet
---

In this project, we read COVID-19 coronavirus data available from [World Health Organization](https://covid19.who.int/) and load it into Google Sheet, where this is one of the data sources used for a [data visualisation work](https://public.tableau.com/profile/peggy.chang#!/vizhome/COVID-19Updates_16082983614350/S1) created for COVID-19 updates on **Tableau Public**.

![Tableau](Tableau.png)



**Tool:**

To load COVID-19 data onto Google Sheet, we use [`pygsheets`](https://pygsheets.readthedocs.io/en/latest/) written by [Nithin Murali](https://github.com/nithinmurali/pygsheets). This is a simple, intuitive Python library to access Google spreadsheets through the Google Sheets API v4.

Although we can use the `importData()` function in Google Sheet itself to obtain data directly from a URL, there are some limitations and it would not work for large datasets.


**Pre-requisite:**

Before running the codes below, we first need to go to [Google Developer Console](https://console.developers.google.com/) to enable the Google Drive API, create a service account and download the private key in a JSON file for account authorization.

We will also need to share the specific Google Sheet with the service account email address.

In [1]:
import pygsheets
import pandas as pd
import datetime as dt

### Authorizing pygsheets.

gc = pygsheets.authorize(service_file='client_secret.json')

In [2]:
### Open an existing Google Sheet by key and clear its content.

sh = gc.open_by_key('1exEhfjPgGzXupYIkkTw36Jxpslf9pbwEiGsxw1xyujw') # WHO_Covid19_GS

wks = sh.worksheet(property='index', value=0)   # value=0 means first tab

wks.clear()

In [3]:
### Get source data from WHO website.

url = "https://covid19.who.int/WHO-COVID-19-global-data.csv"

df = pd.read_csv(url, parse_dates=['Date_reported'])

### Check the number of rows and columns.

df.shape

(97345, 8)

In [4]:
df.describe()

,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
count,97345.000000,9.734500e+04,97345.000000,97345.000000
mean,1116.490246,1.244781e+05,24.645364,3279.255740
std,7519.576166,8.542595e+05,142.643087,18246.997859
min,-32952.000000,0.000000e+00,-514.000000,0.000000
25%,0.000000,6.000000e+00,0.000000,0.000000
50%,3.000000,9.000000e+02,0.000000,15.000000
75%,176.000000,1.758700e+04,3.000000,326.000000
max,402270.000000,2.730950e+07,6409.000000,480464.000000


In [5]:
### Check the column data type.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97345 entries, 0 to 97344
Data columns (total 8 columns):
Date_reported        97345 non-null datetime64[ns]
Country_code         96934 non-null object
Country              97345 non-null object
WHO_region           97345 non-null object
New_cases            97345 non-null int64
Cumulative_cases     97345 non-null int64
New_deaths           97345 non-null int64
Cumulative_deaths    97345 non-null int64
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 5.9+ MB


In [6]:
### Convert the format of 'Date_reported' to dd/mm/yyyy.

df['Date_reported'] = df['Date_reported'].dt.strftime("%d/%m/%Y")

In [7]:
### Check the last 5 records to ensure that we're getting up-to-date data.

df.tail()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
97340,12/02/2021,ZW,Zimbabwe,AFRO,85,34949,18,1382
97341,13/02/2021,ZW,Zimbabwe,AFRO,96,35045,11,1393
97342,14/02/2021,ZW,Zimbabwe,AFRO,59,35104,5,1398
97343,15/02/2021,ZW,Zimbabwe,AFRO,68,35172,2,1400
97344,16/02/2021,ZW,Zimbabwe,AFRO,50,35222,10,1410


In [8]:
df[df["Country_code"]=="SG"].tail()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
79678,12/02/2021,SG,Singapore,WPRO,12,59759,0,29
79679,13/02/2021,SG,Singapore,WPRO,18,59777,0,29
79680,14/02/2021,SG,Singapore,WPRO,9,59786,0,29
79681,15/02/2021,SG,Singapore,WPRO,14,59800,0,29
79682,16/02/2021,SG,Singapore,WPRO,9,59809,0,29


In [9]:
### Write the data from Pandas dataframe to Google Sheet.

wks.set_dataframe(df, start='A1', copy_index=False, copy_head=True, fit=True, escape_formulae=False, nan=0)

Below are the parameters for the [`set_dataframe`](https://pygsheets.readthedocs.io/en/latest/worksheet.html#pygsheets.Worksheet.set_dataframe) method for reference:


- **df** : Pandas dataframe.
- **start** : Address of the top left corner where the data should be added.
- **copy_index** : Copy dataframe index (multi-index supported).
- **copy_head** : Copy header data into the first row.
- **fit** : Resize the worksheet to fit all data inside if necessary.
- **escape_formulae** – Any value starting with an equal sign (=), will be prefixed with an apostrophe (‘) to avoid value being interpreted as a formula.
- **nan** – Value with which NaN values are replaced.